In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [28]:
current_file = '/Users/drewkeller/Desktop/CS/broadbandequity'
blocks = current_file+'/geo/blocks.shp'
community_areas = current_file+'/geo/community_areas.shp'
tracts = current_file+'/geo/tracts.shp'
wards = current_file+'/geo/wards.shp'

In [66]:
target_geography = gpd.read_file(community_areas)
source_geography = gpd.read_file(tracts)

/Users/drewkeller/opt/anaconda3/envs/broadbandequity/lib/python3.8/site-packages/geopandas/array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [69]:
target_geography.head()

,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len,geometry
0,35,35,0.0,0.0,DOUGLAS,0.0,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,36,36,0.0,0.0,OAKLAND,0.0,1.691396e+07,19565.506153,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,37,37,0.0,0.0,FULLER PARK,0.0,1.991670e+07,25339.089750,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."
3,38,38,0.0,0.0,GRAND BOULEVARD,0.0,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
4,39,39,0.0,0.0,KENWOOD,0.0,2.907174e+07,23325.167906,"POLYGON ((-87.59215 41.81693, -87.59215 41.816..."


In [77]:
overlap = gpd.overlay(source_geography,target_geography,how="intersection")

/Users/drewkeller/opt/anaconda3/envs/broadbandequity/lib/python3.8/site-packages/geopandas/array.py:644: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  data[:] = [
/Users/drewkeller/opt/anaconda3/envs/broadbandequity/lib/python3.8/site-packages/geopandas/array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/Users/drewkeller/opt/anaconda3/envs/broadbandequity/lib/python3.8/site-packages/geopandas/array.py:242: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  data[:] = [


In [78]:
print(len(source_geography))
print(len(target_geography))
print(len(overlap))
overlap.head()

801
77
1642


,commarea,commarea_n,countyfp10,geoid10,name10,namelsad10,notes,statefp10,tractce10,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len,geometry
0,44,44.0,031,17031842400,8424,Census Tract 8424,None,17,842400,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"MULTIPOLYGON (((-87.62394 41.72904, -87.62402 ..."
1,54,54.0,031,17031540102,5401.02,Census Tract 5401.02,None,17,540102,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"POLYGON ((-87.61199 41.68544, -87.61195 41.685..."
2,49,49.0,031,17031834000,8340,Census Tract 8340,None,17,834000,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"POLYGON ((-87.63347 41.72267, -87.63347 41.722..."
3,53,53.0,031,17031530100,5301,Census Tract 5301,None,17,530100,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"MULTIPOLYGON (((-87.61251 41.68543, -87.61250 ..."
4,49,49.0,031,17031490902,4909.02,Census Tract 4909.02,None,17,490902,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"POLYGON ((-87.62033 41.69943, -87.62029 41.699..."


In [83]:
overlap['area'] = overlap.area

In [84]:
weighted_mean = lambda x: np.average(x, weights=overlap.loc[x.index, "area"])

In [86]:
values = {'test': {'weighted_mean' : weighted_mean} }

In [92]:
output = overlap.groupby('community').agg(weighted_mean=('test',weighted_mean))

In [93]:
output.head()

,commarea,commarea_n,countyfp10,geoid10,name10,namelsad10,notes,statefp10,tractce10,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len,geometry,area
0,44,44.0,031,17031842400,8424,Census Tract 8424,None,17,842400,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"MULTIPOLYGON (((-87.62394 41.72904, -87.62402 ...",5.775593e-07
1,54,54.0,031,17031540102,5401.02,Census Tract 5401.02,None,17,540102,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"POLYGON ((-87.61199 41.68544, -87.61195 41.685...",2.405867e-11
2,49,49.0,031,17031834000,8340,Census Tract 8340,None,17,834000,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"POLYGON ((-87.63347 41.72267, -87.63347 41.722...",1.065032e-04
3,53,53.0,031,17031530100,5301,Census Tract 5301,None,17,530100,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"MULTIPOLYGON (((-87.61251 41.68543, -87.61250 ...",5.364129e-10
4,49,49.0,031,17031490902,4909.02,Census Tract 4909.02,None,17,490902,49,49,0.0,0.0,ROSELAND,0.0,1.343137e+08,56632.795429,"POLYGON ((-87.62033 41.69943, -87.62029 41.699...",1.293742e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,10,10.0,031,17031810400,8104,Census Tract 8104,Partially outside City Boundary,17,810400,9,9,0.0,0.0,EDISON PARK,0.0,3.163631e+07,25937.226841,"POLYGON ((-87.82141 41.99685, -87.82141 41.996...",1.048528e-13
1636,9,9.0,031,17031090100,901,Census Tract 901,None,17,090100,9,9,0.0,0.0,EDISON PARK,0.0,3.163631e+07,25937.226841,"POLYGON ((-87.82007 42.01501, -87.82007 42.015...",1.013831e-04
1637,9,9.0,031,17031090200,902,Census Tract 902,Partially outside City Boundary,17,090200,9,9,0.0,0.0,EDISON PARK,0.0,3.163631e+07,25937.226841,"POLYGON ((-87.80676 42.00085, -87.80676 42.000...",1.662133e-04
1638,9,9.0,031,17031090300,903,Census Tract 903,Partially outside City Boundary,17,090300,9,9,0.0,0.0,EDISON PARK,0.0,3.163631e+07,25937.226841,"POLYGON ((-87.82134 42.00385, -87.82133 42.004...",5.168843e-05
